In [1]:
from sklearn.datasets import load_iris
import scipy.cluster.hierarchy as sch
import idendrogram
#plot in Altair
from idendrogram.targets.altair import to_altair
import altair as alt

In [2]:
alt.renderers.set_embed_options(actions=False)
data = load_iris(as_frame=True)
linkage_matrix = sch.linkage(data['data'], method='single', metric='euclidean')
threshold = 0.8
flat_clusters = sch.fcluster(linkage_matrix, t=threshold, criterion='distance')
cl_data = idendrogram.ClusteringData(linkage_matrix = linkage_matrix, cluster_assignments = flat_clusters)

# Customizing labels

## Customizing leaf axis labels

### Using `leaf_label_func`

Leaf axis labels are controlled via `leaf_label_func` parameter of  `idd.create_dendrogram` function. To customize labels, pass a callable that returns a desired label for a given linkage ID. 

For example, to display the linkage ID itself:

In [3]:
#instantiate idendrogram object and pass clustering data
idd = idendrogram.idendrogram()
idd.set_cluster_info(cl_data)

#create a dendrogram object
dendrogram = idd.create_dendrogram(
    leaf_label_func = lambda _, linkage_id: str(linkage_id)
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

### Convenience functions

Two convenience labellers are available:

- `idendrogram.callbacks.counts` returns counts of observations represented by the linkage ID. This is the default. 
- `idendrogram.callbacks.cluster_labeller(fmt_string=[..])` returns a callable that returns label only for the first leaf in the cluster, leaving other labels empty. 

#### Leaf labels representing observation counts

In [4]:
dendrogram = idd.create_dendrogram(
    leaf_label_func = idendrogram.callbacks.counts
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

#### Leaf labels shown once per cluster

In [5]:
dendrogram = idd.create_dendrogram(
    leaf_label_func = idendrogram.callbacks.cluster_labeller(
        fmt_string="This is cluster {cluster}"
    )
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

### Angles

Angles of leaf labels are defined in `idendrogram.containers.ClusterNode`. The easiest way to change defaults is by defining a new dataclass with appropriate values and passing it to the `idendrogram()` constructor. 

In [6]:
from idendrogram.containers import AxisLabel
from dataclasses import dataclass

@dataclass
class MyAxisLabel(AxisLabel):
    labelAngle: int = -90
    
idd = idendrogram.idendrogram(axis_label_factory=lambda x:  MyAxisLabel(**x))
idd.set_cluster_info(cl_data)
dendrogram = idd.create_dendrogram(
    leaf_label_func = idendrogram.callbacks.cluster_labeller(
        fmt_string="This is cluster {cluster}"
    ))
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

### Text sizes

Text sizes are not natively integrated into idendrogram. You may be able to change them by manipulating the returned objects directly. E.g., in Altair:

In [7]:
chart = to_altair(dendrogram=dendrogram, height=200, width=629)
chart.configure_axis(
    labelFontSize=15,
    titleFontSize=15
)

alt.LayerChart(...)

## Customizing node labels

#### Using `node_label_func`

Leaf axis labels are controlled via `node_label_func` parameter of  `idd.create_dendrogram` function. To customize labels, pass a callable that returns an appropriate label for a given linkage ID. 

For example, to display the linkage ID itself:

In [8]:
from idendrogram.containers import ClusterNode

#this is to change default node color/radius for better readability
@dataclass
class MyNode(ClusterNode):
    fillcolor: str = 'black'
    _default_leaf_radius: float = 10    
    radius: float = 20
    
idd = idendrogram.idendrogram(node_factory=lambda x: MyNode(**x))
idd.set_cluster_info(cl_data)

# pass node_label_func a callable that returns linkage_id
dendrogram = idd.create_dendrogram(
    node_label_func= lambda _, linkage_id: str(linkage_id),
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

### Displaying cluster IDs

By default, idendrogram uses `idendrogram.callbacks.cluster_id_if_cluster` which returns flat cluster ID if the linkage ID corresponds to the flat cluster and empty label otherwise. All callback functions share the same signature, so you can get creative. Here's how you could combine it with `idendrogram.callbacks.counts` to show cluster ID and the number of observations in the cluster.

In [9]:
def custom_label(cl, id):
    cluster_id = idendrogram.callbacks.cluster_id_if_cluster(cl, id)
    obs_count = idendrogram.callbacks.counts(cl, id)

    return f"""C{cluster_id} ({obs_count})""" if cluster_id != "" else ""

# pass node_label_func a callable that returns linkage_id
dendrogram = idd.create_dendrogram(
    node_label_func= custom_label
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

## Customizing tooltips

### Using `node_hover_func`

Leaf axis labels are controlled via `node_hover_func` parameter of  `idd.create_dendrogram` function. To customize labels, pass a callable that returns a dictionary of key:value pairs that will be displayed in a tooltip for a given linkage ID. 

For example, to display the linkage ID itself:

In [10]:
idd = idendrogram.idendrogram()
idd.set_cluster_info(cl_data)

dendrogram = idd.create_dendrogram(
    node_hover_func= lambda _, id: {'linkage ID': id}
)
to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)

idendrogram uses `idendrogram.callbacks.default_hover` that displays the linkage ID and the # of observations associated with this linkage ID by default. 

### Customizing via postprocessing 

You can also rely on post-processing of dendrograms to customize tooltips. Here's an example using two of available callback functions.

In [15]:
idd = idendrogram.idendrogram()
idd.set_cluster_info(cl_data)

dendrogram = idd.create_dendrogram(
    node_hover_func= lambda _, id: {'linkage ID': id}
)

for n in dendrogram.nodes:
    n.hovertext = {
        'Cluster': idendrogram.callbacks.cluster_assignments(cl_data, n.id),
        "Node type": n.type,
        "Node color": n.fillcolor,
        "Original tooltip": n.hovertext,
        "Number of observations": idendrogram.callbacks.counts(cl_data, n.id)
    }

to_altair(dendrogram=dendrogram, height=200, width=629)

alt.LayerChart(...)